# Main code

## Import libraries, create class objects, and create functions/classes

In [1]:
# import libraries
import scrapy
from scrapy.crawler import CrawlerRunner
import requests
import numpy as np
import pandas as pd

# import reactor restart in order to allow running the spider many times
from crochet import setup, wait_for
setup()
@wait_for(50)
def run_spider(my_spider):
    crawler = CrawlerRunner()
    d = crawler.crawl(my_spider)
    return d

# check wheter the element exist or not
def check_element(path):
    """to check the element based on xpath/css locator notation of HTML code.
       if there is no element then returns NaN, else returns a value based on the corresponding element we want to extract
    """
    if path:
        output = path
    else:
        output = np.nan
    return output

# check wheter the room type exist or not
def check_room_type(path):
    if path:
        return 'yes'
    else:
        return 'no'

In [2]:
# create "Hyperlink_Scraper"
class Hyperlink_Scraper(scrapy.Spider):
    name = "Hyperlink_Scraper"

    # instantiate start_requests() method from scrapy to crawl the 'main_url'
    def start_requests(self):
        yield scrapy.Request(url = main_url, callback = self.parse_hyperlink)
                             
    # get hyperlinks from the 'main_url' then go to the next page of the 'main_url' if the next page is exist
    def parse_hyperlink(self, response):
        urls_to_follow = response.xpath('//div[@class="media property-item search"]').css('a.hover-effect::attr(href)').extract()
        for url in urls_to_follow:
            list_of_urls.append(url)

In [3]:
# create "Kost_Scraper"
class Kost_Scraper(scrapy.Spider):
    name = 'Kost_Scraper'
    
    # instantiate start_requests() method from scrapy to crawl the 'kost_url'
    def start_requests(self):
        yield scrapy.Request(url = kost_url, callback = self.parse_kost_url)
        
    # get hyperlinks from the 'main_url' then go to the next page of the 'main_url' if the next page is exist
    def parse_kost_url(self, response):
        data.append({
            'link_kost': response.url,
            'nama_kost': response.css('h1.listing-title::text').extract_first().strip(),
            'alamat_kost_1': response.css('div.block-body address.item-address::text').extract_first().strip(),
            'alamat_kost_2': response.css('div.block-body address.sec-address::text').extract_first().strip(),
            'latitude': response.xpath('//div[@id="map-section"]//div[@id="banner-map"]//@data-lat').extract_first(),
            'longitude': response.xpath('//div[@id="map-section"]//div[@id="banner-map"]//@data-long').extract_first(),
            'jenis_kost': response.css('div.col-sm-2.col-md-3.col-xs-6.no-r-pad.gender-box > strong::text').extract_first().strip(),
            'luas_kamar': response.css('div.col-sm-8.col-md-6.col-xs-12 strong::text').extract_first(),
            'fasilitas_gedung': response.css('div.row.mob-bg-white div.block-right li::text').extract(),
            'sekitar_gedung': response.css('div.row div.block-right.facilities-list li::text').extract(),
            'harga_mulai': response.css('div.sidebar-price::text').extract_first(),
            'tipe_kamar': check_room_type(response.css('div.title-head-left h2.title.roomtitle::text').extract()),
            'tipe_kamar (nama_kamar)': response.css('div.title-head-left h2.title.roomtitle::text').extract(),
            'tipe_kamar (luas_kamar)': response.xpath('//*[contains(@id,"room")]/div/div[2]/div[1]/div/address/text()').extract(),
            'tipe_kamar (harga_kamar)': response.xpath('//*[contains(@id,"room")]/div/div[3]/div/div/div/div/span/text()').extract(),
            'tipe_kamar (spesifikasi_kamar)': [response.xpath('//*[@id="room{}"]'.format(i+1)).xpath('.//ul[@class="item-amenities"]//span/text()').extract() for i in range(len(response.css('div.title-head-left h2.title.roomtitle::text').extract()))],
            'tipe_kamar (fasilitas_kamar)': [response.xpath('//*[@id="room{}"]'.format(i+1)).css('ul.detail-list-disc.no-pad li ::text').extract() for i in range(len(response.css('div.title-head-left h2.title.roomtitle::text').extract()))]
        })

## Extract hyperlinks which contain the main pages

In [4]:
### get all main urls ###

# Bandung (68 pages)
kost_bandung = []
for i in range(68):
    kost_bandung.append('https://infokost.id/search-results/page/{}/?location_search=Bandung%2C%20Bandung%20City%2C%20West%20Java%2C%20Indonesia&search_city=bandung&search_area=bandung-city&search_country=indonesia&lat=-6.9174639&lng=107.6191228&radius=5&sortby'.format(i+1))
print('sample Bandung:', kost_bandung[-1], '\n')

# Jogja (20 pages)
kost_yogyakarta = []
for i in range(20):
    kost_yogyakarta.append('https://infokost.id/search-results/page/{}/?location_search=Yogyakarta%2C%20Yogyakarta%20City%2C%20Special%20Region%20of%20Yogyakarta%2C%20Indonesia&search_city=yogyakarta&search_area=yogyakarta-city&search_country=indonesia&lat=-7.7955798&lng=110.3694896&radius=5&sortby'.format(i+1))
print('sample Yogyakarta:', kost_yogyakarta[-1], '\n')

# Bali (14 pages)
kost_bali = []
for i in range(14):
    kost_bali.append('https://infokost.id/search-results/page/{}/?location_search=Denpasar%2C%20Denpasar%20City%2C%20Bali%2C%20Indonesia&search_city=denpasar&search_area=denpasar-city&search_country=indonesia&lat=-8.670458199999999&lng=115.2126293&radius=5&sortby'.format(i+1))
print('sample Bali:', kost_bali[-1], '\n')

# Jakarta Barat (105 pages)
kost_jakbar = []
for i in range(105):
    kost_jakbar.append('https://infokost.id/search-results/page/{}/?location_search=Jakarta%20Barat%2C%20West%20Jakarta%20City%2C%20Jakarta%2C%20Indonesia&search_city=west-jakarta&search_area=west-jakarta-city&search_country=indonesia&lat=-6.167430899999999&lng=106.7637239'.format(i+1))
print('sample Jakarta Barat:', kost_jakbar[-1], '\n')

# Jakarta Pusat (105 pages)
kost_jakpus = []
for i in range(105):
    kost_jakpus.append('https://infokost.id/search-results/page/{}/?location_search=Jakarta%20Pusat%2C%20Central%20Jakarta%20City%2C%20Jakarta%2C%20Indonesia&search_city=central-jakarta&search_area=central-jakarta-city&search_country=indonesia&lat=-6.1805113&lng=106.8283831'.format(i+1))
print('sample Jakarta Pusat:', kost_jakpus[-1], '\n')

# Jakarta Selatan (109 pages)
kost_jaksel = []
for i in range(109):
    kost_jaksel.append('https://infokost.id/search-results/page/{}/?location_search=Jakarta%20Selatan%2C%20South%20Jakarta%20City%2C%20Jakarta%2C%20Indonesia&search_city=south-jakarta&search_area=south-jakarta-city&search_country=indonesia&lat=-6.2614927&lng=106.8105998'.format(i+1))
print('sample Jakarta Selatan:', kost_jaksel[-1], '\n')

# Jakarta Timur (104 pages)
kost_jaktim = []
for i in range(104):
    kost_jaktim.append('https://infokost.id/search-results/page/{}/?location_search=Jakarta%20Timur%2C%20East%20Jakarta%20City%2C%20Jakarta%2C%20Indonesia&search_city=east-jakarta&search_area=east-jakarta-city&search_country=indonesia&lat=-6.2250138&lng=106.9004472'.format(i+1))
print('sample Jakarta Timur:', kost_jaktim[-1], '\n')

# Jakarta Utara (99 pages)
kost_jakut = []
for i in range(99):
    kost_jakut.append('https://infokost.id/search-results/page/{}/?location_search=Jakarta%20Utara%2C%20North%20Jakarta%20City%2C%20Jakarta%2C%20Indonesia&search_city=north-jakarta&search_area=north-jakarta-city&search_country=indonesia&lat=-6.155405699999999&lng=106.8926634'.format(i+1))
print('sample Jakarta Utara:', kost_jakut[-1], '\n')

# Depok (40 pages)
kost_depok = []
for i in range(40):
    kost_depok.append('https://infokost.id/search-results/page/{}/?location_search=Depok%2C%20West%20Java%2C%20Indonesia&search_city=Depok%20City&search_area=depok-city&search_country=indonesia&lat=-6.4024844&lng=106.7942405'.format(i+1))
print('sample Depok:', kost_depok[-1], '\n')

# Tangerang (47 pages)
kost_tangerang = []
for i in range(47):
    kost_tangerang.append('https://infokost.id/search-results/page/{}/?location_search=Tangerang%2C%20Tangerang%20City%2C%20Banten%2C%20Indonesia&search_city=tangerang&search_area=tangerang-city&search_country=indonesia&lat=-6.1701796&lng=106.6403236'.format(i+1))
print('sample Tangerang:', kost_tangerang[-1], '\n')

sample Bandung: https://infokost.id/search-results/page/68/?location_search=Bandung%2C%20Bandung%20City%2C%20West%20Java%2C%20Indonesia&search_city=bandung&search_area=bandung-city&search_country=indonesia&lat=-6.9174639&lng=107.6191228&radius=5&sortby 

sample Yogyakarta: https://infokost.id/search-results/page/20/?location_search=Yogyakarta%2C%20Yogyakarta%20City%2C%20Special%20Region%20of%20Yogyakarta%2C%20Indonesia&search_city=yogyakarta&search_area=yogyakarta-city&search_country=indonesia&lat=-7.7955798&lng=110.3694896&radius=5&sortby 

sample Bali: https://infokost.id/search-results/page/14/?location_search=Denpasar%2C%20Denpasar%20City%2C%20Bali%2C%20Indonesia&search_city=denpasar&search_area=denpasar-city&search_country=indonesia&lat=-8.670458199999999&lng=115.2126293&radius=5&sortby 

sample Jakarta Barat: https://infokost.id/search-results/page/105/?location_search=Jakarta%20Barat%2C%20West%20Jakarta%20City%2C%20Jakarta%2C%20Indonesia&search_city=west-jakarta&search_area=west

In [5]:
%%time
### extract hyperlinks in which contain kost urls from main urls ###

# list_of_urls ---> list to save extracted hyperlinks
list_of_urls = []

# kost_all ---> list of main urls
kost_all = kost_bandung + kost_yogyakarta + kost_bali + kost_jakbar + kost_jakpus + kost_jaksel + kost_jaktim + kost_jakut + kost_depok + kost_tangerang
print('total urls in which the hyperlinks will be extracted in each of that url:', len(kost_all))

from tqdm import tqdm
for i in tqdm(range(len(kost_all))):
    main_url = kost_all[i]
    run_spider(Hyperlink_Scraper)

total urls in which the hyperlinks will be extracted in each of that url: 711


100%|████████████████████████████████████████████████████████████████████████████████| 711/711 [36:46<00:00,  3.10s/it]

Wall time: 36min 46s


In [6]:
print('The number of extracted hyperlinks:', len(list_of_urls))

list_of_urls_no_duplicates = list(set(list_of_urls))
print('The number of extracted hyperlinks with no duplicate hyperlinks:', len(list_of_urls_no_duplicates))

pd.DataFrame(list_of_urls_no_duplicates).to_csv('dataset/Kost URLs Data.csv', index = False, header = ['kost_url'])

The number of extracted hyperlinks: 11290
The number of extracted hyperlinks with no duplicate hyperlinks: 3436


## Parse kost unit information from main pages

In [7]:
kost_urls = pd.read_csv('dataset/Kost URLs Data.csv')['kost_url'].tolist()
len(kost_urls)

3436

In [8]:
%%time
### parse kost entities information from kost urls ###

# data ---> list to save extracted information from each kost url
data = []

from tqdm import tqdm
for i in tqdm(range(len(kost_urls))):
    kost_url = kost_urls[i]
    run_spider(Kost_Scraper)

100%|████████████████████████████████████████████████████████████████████████████| 3436/3436 [4:45:48<00:00,  4.99s/it]

Wall time: 4h 45min 48s


In [9]:
pd.DataFrame(data).to_csv('dataset/Kost Data (Raw).csv', index = False)

import json
with open('dataset/Kost Data (Raw).json', 'w') as fout:
    json.dump(data, fout)

In [10]:
kost_raw = pd.read_csv('dataset/Kost Data (Raw).csv')
kost_raw

,link_kost,nama_kost,alamat_kost_1,alamat_kost_2,latitude,longitude,jenis_kost,luas_kamar,fasilitas_gedung,sekitar_gedung,harga_mulai,tipe_kamar,tipe_kamar (nama_kamar),tipe_kamar (luas_kamar),tipe_kamar (harga_kamar),tipe_kamar (spesifikasi_kamar),tipe_kamar (fasilitas_kamar)
0,https://infokost.id/listing/ruoptions-caraka-i...,Caraka Inn Pesanggrahan,"Jl. Pesanggrahan V No.25, RT.8/RW.7, South Pet...","PETUKANGAN SELATAN, PESANGGRAHAN, ...",-6.2400225,106.754748,Kost Campur,NaN,"[' Area parkir', ' CCTV', ' Ruang santai / rua...","['\xa0\xa0Supermarket', '\xa0\xa0Sekolah', '\x...",Rp 1.250.000/bulan,yes,"[' Pocket Single', ' Pocket Double']","['Luas kamar: 6.5 -- 7.5m', 'Luas kamar: 6.5 -...","[' Rp 1.250.000', ' ', ' Rp 1.250.000', ' ']","[['1 Tempat tidur (100x200) 1 orang', 'Tidak t...","[['AC', 'Lemari', 'TV'], ['AC', 'Lemari', 'TV']]"
1,https://infokost.id/listing/kost-pondok-latifa...,Kost Pondok Latifa Coblong Bandung,jl titimplik no18 rt07 rw09 kelurahan sadang s...,"GEGERKALONG, SUKASARI, KOT...",-6.896961223,107.621984,Kost Putri,Luas Kamar: 8--10 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 700.000/bulan,no,[],[],[],[],[]
2,https://infokost.id/listing/kost-the-loft-cobl...,Kost The Loft Coblong Bandung,jln. cisitu baru no. 60b. dago. coblong. bandung,"BATUNUNGGAL, BANDUNG KIDUL, ...",-6.88379416,107.612552,Kost Putra,Luas Kamar: 13 -- 15 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 2.100.000/bulan,no,[],[],[],[],[]
3,https://infokost.id/listing/kost-perum-alamand...,Kost Perum Alamanda Coblong Bandung,"Jl. Tubagus Ismail Raya No.60, Sekeloa, Kecama...","SADANG SERANG, COBLONG, KO...",-6.885143026,107.622391,Kost Campur,Luas Kamar: 10 -- 12 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 1.300.000/bulan,no,[],[],[],[],[]
4,https://infokost.id/listing/kost-nijisi-house-...,Kost Nijisi House Tipe A Coblong Bandung,"Jl. Kanayakan Baru 23, Dago, Coblong, Bandun","SARIJADI, SUKASARI, KOTA B...",-6.878314919,107.617798,Kost Putri,Luas Kamar: 8--10 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 1.750.000/bulan,no,[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3429,https://infokost.id/listing/kost-ibu-sumarni-b...,Kost Ibu Sumarni Bandara Husein Nurtanio Bandung,"Jalan Pajajaran Dalam No.198/72, RT.01/RW.03, ...","LEBAK GEDE, COBLONG, KOTA ...",-6.906038881,107.586025,Kost Campur,Luas Kamar: 8--10 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 900.000/bulan,no,[],[],[],[],[]
3430,https://infokost.id/listing/kost-syifa-sukajad...,Kost syifa Sukajadi Bandung,Jln karang tineng dalam gg H gojali rt 05/05 n...,"ARJUNA, CICENDO, KOTA BAND...",-6.8920454,107.592725,Kost Campur,Luas Kamar: 15--20 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 700.000/bulan,no,[],[],[],[],[]
3431,https://infokost.id/listing/kost-griya-narayan...,Kost Griya Narayana Prapanca Tipe A Mantrijero...,"Jl Prapanca no.24 Rt 57 Gedongkiwo, Mantrijero...","GEDONGKIWO, MANTRIJERON, K...",-7.8172626,110.353949,Kost Putri,Luas Kamar: 10 -- 12 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 850.000/bulan,no,[],[],[],[],[]
3432,https://infokost.id/listing/kost-amira25-cicen...,Kost Amira25 Cicendo Bandung,"jalan junjunan dalam 1, No. 25, kecamatan Cice...","SUKAGALIH, SUKAJADI, KOTA ...",-6.895719968,107.587896,Kost Campur,Luas Kamar: 15--20 m,[],"['\xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' ...",Rp 850.000/bulan,no,[],[],[],[],[]


# Misc

In [11]:
from scrapy.http.response.text import TextResponse
def create_response(url):
    req = scrapy.Request(url = url)   # get 'Request' object
    url = req.url                     # get url link from 'Request' object
    html = requests.get(url).content  # get 'Respone' object then extract html code
    response = TextResponse(url = url, body = html, encoding = 'utf-8') # create 'TextResponse' object from url and HTMl code
    return response, html

def extract_information(url):
    response, html = create_response(url)
    print('link_kost:', response.url)
    print('nama_kost:', response.css('h1.listing-title::text').extract_first().strip())
    print('alamat_kost_1:', response.css('div.block-body address.item-address::text').extract_first().strip())
    print('alamat_kost_2:', response.css('div.block-body address.sec-address::text').extract_first().strip())
    print('latitude:', response.xpath('//div[@id="map-section"]//div[@id="banner-map"]//@data-lat').extract_first())
    print('longitude:', response.xpath('//div[@id="map-section"]//div[@id="banner-map"]//@data-long').extract_first())
    print('jenis_kost:', response.css('div.col-sm-2.col-md-3.col-xs-6.no-r-pad.gender-box > strong::text').extract_first().strip())
    print('luas_kamar:', response.css('div.col-sm-8.col-md-6.col-xs-12 strong::text').extract_first())
    print('fasilitas_gedung:', response.css('div.row.mob-bg-white div.block-right li::text').extract())
    print('sekitar_gedung:', response.css('div.row div.block-right.facilities-list li::text').extract())
    print('harga_mulai:', response.css('div.sidebar-price::text').extract_first(), '\n')
    print('tipe_kamar:', check_room_type(response.css('div.title-head-left h2.title.roomtitle::text').extract()), '\n')
    print('tipe_kamar (nama_kamar):\n', response.css('div.title-head-left h2.title.roomtitle::text').extract(), '\n')
    print('tipe_kamar (luas_kamar):\n', response.xpath('//*[contains(@id,"room")]/div/div[2]/div[1]/div/address/text()').extract(), '\n')
    print('tipe_kamar (harga_kamar):\n', response.xpath('//*[contains(@id,"room")]/div/div[3]/div/div/div/div/span/text()').extract(), '\n')
    print('tipe_kamar (spesifikasi_kamar):\n', [response.xpath('//*[@id="room{}"]'.format(i+1)).xpath('.//ul[@class="item-amenities"]//span/text()').extract() for i in range(len(response.css('div.title-head-left h2.title.roomtitle::text').extract()))], '\n')
    print('tipe_kamar (fasilitas_kamar):\n', [response.xpath('//*[@id="room{}"]'.format(i+1)).css('ul.detail-list-disc.no-pad li ::text').extract() for i in range(len(response.css('div.title-head-left h2.title.roomtitle::text').extract()))], '\n')
    return response

In [12]:
# scrape the main page
response, html = create_response(kost_bandung[33])
link_extracted = response.xpath('//div[@class="media property-item search"]').css('a.hover-effect::attr(href)').extract()
print('The main page is', kost_bandung[0], '\n')
print('Other links from the main page to be scrapped:', len(link_extracted))
display(link_extracted)

The main page is https://infokost.id/search-results/page/1/?location_search=Bandung%2C%20Bandung%20City%2C%20West%20Java%2C%20Indonesia&search_city=bandung&search_area=bandung-city&search_country=indonesia&lat=-6.9174639&lng=107.6191228&radius=5&sortby 

Other links from the main page to be scrapped: 16


['https://infokost.id/listing/kost-rumah-ks-8-tipe-ii-coblong-bandung/',
 'https://infokost.id/listing/kost-rumah-ks-8-coblong-bandung/',
 'https://infokost.id/listing/kost-rumah-kirana-ciheulang-tipe-e-coblong-bandung/',
 'https://infokost.id/listing/kost-rumah-ke-2-tipe-a-cicendo-bandung-wetan-bandung/',
 'https://infokost.id/listing/kost-rumah-cipaganti-mulia-hospitality-sukajadi-bandung/',
 'https://infokost.id/listing/kost-rumah-cihaur-22-dago-bandung/',
 'https://infokost.id/listing/kost-rumah-bumi-hegar-pasteur-bandung/',
 'https://infokost.id/listing/kost-rumah-bu-imas-tipe-a-sukajadi-bandung/',
 'https://infokost.id/listing/kost-rumah-boscha-sukajadi-bandung/',
 'https://infokost.id/listing/kost-rumah-berkat-bandung/',
 'https://infokost.id/listing/kost-rumah-bapak-hada-coblong-bandung/',
 'https://infokost.id/listing/kost-rumah-bangbayang-coblong/',
 'https://infokost.id/listing/kost-rumah-asri-tipe-a-cicendo-bandung/',
 'https://infokost.id/listing/kost-rumah-amira-coblong-b

In [13]:
# example of scrapped page (1)
response = extract_information('https://infokost.id/listing/kost-exclusive-bergengsi/')

link_kost: https://infokost.id/listing/kost-exclusive-bergengsi/
nama_kost: Kost Exclusive Bergengsi
alamat_kost_1: Jl. Lombok No. 19 Merdeka Andir Bandung
alamat_kost_2: MERDEKA,           SUMUR BANDUNG,           KOTA BANDUNG,           DKI JAKARTA           40113
latitude: -6.909863
longitude: 107.618798
jenis_kost: Kost Campur
luas_kamar: None
fasilitas_gedung: [' Area makan', ' Area parkir', ' Cleaning service', ' Kulkas', ' Laundry', ' Ruang santai / ruang tamu', ' Tempat jemur pakaian', ' Wifi']
sekitar_gedung: ['\xa0\xa0Tol', ' \xa0\xa0Supermarket', ' \xa0\xa0Sekolah', ' \xa0\xa0Rumah Sakit', ' \xa0\xa0Rumah Makan', ' \xa0\xa0Mall', ' \xa0\xa0Kampus', ' \xa0\xa0Airport']
harga_mulai: Rp 4.000.000/bulan 

tipe_kamar: yes 

tipe_kamar (nama_kamar):
 [' Presidential Suite Room', ' Deluxe Queen BedRoom', ' Family Suite Room', ' Superior Twin BedRoom', ' Superior Queen BedRoom'] 

tipe_kamar (luas_kamar):
 ['Luas kamar: 24m', 'Luas kamar: 28m', 'Luas kamar: 18m', 'Luas kamar: 18m', 

In [14]:
# example of scrapped page (2)
response = extract_information('https://infokost.id/listing/purabaya-residence-2/')

link_kost: https://infokost.id/listing/purabaya-residence-2/
nama_kost: Purabaya Residence
alamat_kost_1: Jl. Purabaya No.31/66, Arjuna, Kec. Cicendo, Kota Bandung, Jawa Barat 40172
alamat_kost_2: ARJUNA,           CICENDO,           KOTA BANDUNG,           JAWA BARAT           40172
latitude: -6.907248934038263
longitude: 107.59545194420441
jenis_kost: Kost Campur
luas_kamar: Luas Kamar: 8-15 m
fasilitas_gedung: [' Area makan', ' Area parkir', ' CCTV', ' Dapur', ' Ruang santai / ruang tamu', ' Wifi']
sekitar_gedung: ['Istana Plaza', 'Yogya supermarket', 'Living Plaza', 'KFC, Richeese Factory, Badung', 'RS Melinda', 'Paskal 23', 'GOR Pajajaran.\n', '– 4 menit naik kendaraan ke Bandara Husein Sastranegara\n', '– 6 menit naik kendaraan ke Stasiun Bandung\n', '– 9 menit naik kendaraan ke Universitas Maranatha\n', '– 9 menit naik kendaraan ke ITB\n', '– 15 menit naik kendaraan ke Universitas Pendidikan Indonesia\n', '– 15 menit naik kendaraan ke Unpar\n', '– dekat dengan jalan Pajajaran, C